In [11]:
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 22})
import numpy as np
import itertools
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
from parse_cube_files import CUBE
import alchemy_tools as at

# Load cube-files

In [148]:
'/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_000227/cube-files/ve_38.cube'
base_path = '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/'
compounds = ['dsgdb9nsd_003585/cube-files/','dsgdb9nsd_002967/cube-files/', 'dsgdb9nsd_001212/cube-files/', 'dsgdb9nsd_003886/cube-files/', 'dsgdb9nsd_000227/cube-files/', 'dsgdb9nsd_003712/cube-files/']
# compounds = ['dsgdb9nsd_003335/cube-files/', 'dsgdb9nsd_009030/cube-files/', 'dsgdb9nsd_021325/cube-files/']
lam_vals = ['ve_38.cube', 've_30.cube', 've_23.cube', 've_15.cube', 've_8.cube']

In [149]:
paths = []
# build paths to cube-files
for com in compounds:
    paths_compound = []
    for lam in lam_vals:
        paths_compound.append(base_path + com + lam)
    paths.append(paths_compound)

In [150]:
cubes = []
for p in paths:
    cubes_compound = []
    for cube in p:
        cubes_compound.append(CUBE(cube))
    cubes.append(cubes_compound)
        

In [68]:
new_cubes = []
for lam in lam_vals:
    new_cubes.append(CUBE(base_path + compounds[-1] + lam))
cubes.append(new_cubes)

# Plot densities

In [161]:
# plot density 1D projections
# errors = ['Error = 0.005 Ha', 'Error = 0.05 Ha', 'Error = 1.16 Ha', 'Error = 2.34 Ha', 'Error = 3.70 Ha', 'Error = 4.08 Ha']

sa = (1,2)
fig, ax = plt.subplots(3,2)
idc = list(itertools.product([0,1,2],[0,1]))
for idx, com in enumerate(cubes):
    for cube_file in com:
        x = np.linspace(0, 20, 175)
        ax[idc[idx]].plot(x, cube_file.project(sa))#, label=label[idx])
#         ax[idc[idx]].set_title(errors[idx])
        if idx == 2:
            ax[idc[idx]].set_xlabel('Cell coordinate (Ang)')
        ax[idc[idx]].set_xlim(0.5, 19.5)
        if idx ==0:
            ax[idc[idx]].set_ylabel(r'Projected Density $\rho(x_0)$ (Bohr$^{-3}$) ')



In [167]:
plt.plot(np.linspace(0,20,175),cubes[0][1].project((0,1)))
plt.xlabel('Cell coordinate (Ang)')
plt.ylabel(r'Projected Density $\rho(x_0)$ (Bohr$^{-3}$) ')
plt.xlim(0, 20)

(0, 20)

In [151]:
change_dens = []
for com in cubes:
    ve30 = com[[0][1]]
    delta = []
    for perm in itertools.combinations([0,1,2], 2):
        # project
        pr = ve30.project(perm)
        # divide in middle
        div_idx = int(len(pr)/2)
        # calculate increase towards border
        delta1 = pr[0] - np.amin(pr[0:div_idx])
        delta2 = pr[len(pr)-1] - np.amin(pr[div_idx:len(pr)-1])
        delta.extend([delta1, delta2])
    mean = np.array(delta).mean()
    std = np.array(delta).std()
    change_dens.append([mean, std])

In [145]:
error128 = [0.0004919845368407394, 0.24519761105562007, 0.4078284198416049]
mean128 = np.array(change_dens)[:,0]
std128 = np.array(change_dens)[:,1]

In [152]:
error143 = np.array([0.004940390267178882, 0.09632850314161116, 0.23383183313433378, 0.2315427443636624])
mean143 = np.array(change_dens)[1:5,0]
std143 = np.array(change_dens)[1:5,1]


In [160]:
plt.errorbar(error128, mean128, yerr=std128, marker='.', linestyle='', label = 'd = 128')
plt.errorbar(error143, mean143, yerr=std143, marker='.', linestyle='', label = 'd = 143')
plt.legend(loc='lower right')
plt.xlabel('Mean error per molecule (Ha)')
plt.ylabel(r'$\Delta \rho(\lambda = 0.8)$')
plt.show()